In [ ]:
from importlib import reload
import copy_script
import GameDictConverter as gdc
import matplotlib.pyplot as plt

In [ ]:
# Use this if you have edited copy_script or GameDictConverter while this notebook was running
reload(copy_script)
reload(gdc)

In [ ]:
# Sets the last played game as the currently observed game
# This only detects games where at least one save has taken place so you need to play to at least one autosave!
copy_script.init(autoselect_newest=True)

In [ ]:
# Starts the copying process *in a new thread*
copy_script.start_copy()

In [ ]:
# Sets a flag to stop the copy thread
copy_script.stop_copy()

In [ ]:
# Creates a timeline like dictionary for the current and another save
meta_dict = gdc.load_all_game_dicts(copy_script.save_game)
other_save_name = "cirrulanstarcooperative2_2143221768"
meta_dict2 = gdc.load_all_game_dicts(other_save_name)

In [ ]:
# Extends the basic game dicts (adds GDP etc.)
extended, res_dict = gdc.extend_empire_base_array(*gdc.create_empire_base_array(copy_script.save_game,"0",meta_dict=meta_dict))
extended2, res_dict2 = gdc.extend_empire_base_array(*gdc.create_empire_base_array(other_save_name,"0",meta_dict=meta_dict2))

In [ ]:
# Plot the GDP for the main player empire in the previously loaded saves
import matplotlib.pyplot as plt
time_conv = gdc.get_time_conversion(copy_script.save_game)
time_conv2 = gdc.get_time_conversion(other_save_name)
plt.plot(time_conv,extended[:,-2,0], label="Without research & unity")
plt.plot(time_conv,extended[:,-1,0], label="With research & unity")
plt.plot(time_conv2,extended2[:,-2,0], label="Without research & unity")
plt.plot(time_conv2,extended2[:,-1,0], label="With research & unity")
plt.xlabel("Time")
plt.ylabel("BIP [EC-equivalent]")
plt.legend()
plt.savefig("BIP.png",dpi=600)

In [ ]:
# Neater printing for dicts, there is probably some library for it...
def nice_print(val, label="", depth=0):
    if isinstance(val, dict):
        print(" "*4*depth + label + ":{ ")
        for key,value in val.items():
            nice_print(value, key, depth+1)
        print(" "*4*depth + "} ")
    else:
        print(" "*4*depth + label + ": " + val)

In [ ]:
# Some other basic plots and showcase of the generate_time_series function
nice_print(meta_dict['gamestate_2246.01.01.json']["country"]["0"]["tech_status"])
time_conv = gdc.get_time_conversion(copy_script.save_game)

#print(meta_dict['gamestate_2246.01.01.json']["country"]["0"].keys())
employable_pops = gdc.generate_time_series(meta_dict, ["country","0","employable_pops"], conversion=lambda x: int(x))
military_power = gdc.generate_time_series(meta_dict, ["country","0","military_power"], conversion=lambda x: float(x))
economy_power = gdc.generate_time_series(meta_dict, ["country","0","economy_power"], conversion=lambda x: float(x))
victory_rank = gdc.generate_time_series(meta_dict, ["country","0","victory_rank"], conversion=lambda x: int(x))
victory_score = gdc.generate_time_series(meta_dict, ["country","0","victory_score"], conversion=lambda x: float(x))
sapient = gdc.generate_time_series(meta_dict, ["country","0","sapient"], conversion=lambda x: int(x))
num_upgraded_starbase = gdc.generate_time_series(meta_dict, ["country","0","num_upgraded_starbase"], conversion=lambda x: int(x))
starbase_capacity = gdc.generate_time_series(meta_dict, ["country","0","starbase_capacity"], conversion=lambda x: int(x))

plt.plot(time_conv,economy_power)
plt.show()
plt.plot(time_conv,military_power)

In [ ]:
# Some more complex usages, I'm not explaining it as it's a mess and I wrote it months (only god knows what it does now)
conv,_ = gdc.get_all_empires(meta_dict)
time_conv = gdc.get_time_conversion(copy_script.save_game)
l = conv.items()
print(conv)
#l = {k:v for k,v in conv.items() if (v != k) and ("_" not in k) and (len(k)>2)}.items()
l = {k:v for k,v in conv.items() if (int(v)<50) and ("_" not in k) and (len(k)>0)}.items()
for name, empire_id in l:
    arr, res_dict = gdc.create_empire_base_array(copy_script.save_game, empire_id, meta_dict=meta_dict)
    extended, ex_res_dict = gdc.extend_empire_base_array(arr, res_dict)
    if empire_id == "0":
        plt.plot(time_conv,extended[:,-1,2], label=name, color="black")
    else:
        plt.plot(time_conv,extended[:,-1,2], label=name, alpha=0.5)
plt.xlabel("Year")
plt.ylabel("NIP")
plt.show()

for name, empire_id in l:
    arr, res_dict = gdc.create_empire_base_array(copy_script.save_game, empire_id, meta_dict=meta_dict)
    extended, ex_res_dict = gdc.extend_empire_base_array(arr, res_dict)
    if empire_id == "0":
        plt.plot(time_conv,extended[:,-1,0], label=name, color="black")
        plt.plot(time_conv,extended[:,-2,0], label=name, color="black", linestyle="--")
    else:
        plt.plot(time_conv,extended[:,-1,0], label=name, alpha=0.5)
        plt.plot(time_conv,extended[:,-2,0], label=name, alpha=0.5, linestyle="--")
plt.xlabel("Year")
plt.ylabel("BIP")
plt.show()

print(res_dict)

for name, empire_id in l:
    arr, res_dict = gdc.create_empire_base_array(copy_script.save_game, empire_id, meta_dict=meta_dict)
    extended, ex_res_dict = gdc.extend_empire_base_array(arr, res_dict)
    if empire_id == "0":
        plt.plot(time_conv,extended[:,5,0]+extended[:,6,0]+extended[:,7,0], label=name, color="black")
    else:
        plt.plot(time_conv,extended[:,5,0]+extended[:,6,0]+extended[:,7,0], label=name, alpha=0.5)
plt.xlabel("Year")
plt.ylabel("Research")
plt.show()